In [14]:
import pandas as pd
import numpy as np
import xarray as xr

import hurdat as h
import tc_functions as fun
import plotting_functions as tcplt

import os

In [15]:
# Get Kim's raw data and import while applying different RI cutoffs
nal_15 = pd.read_table("data/DTL_NAL_2015.txt", delimiter = ' ')
nal_16 = pd.read_table("data/DTL_NAL_2016.txt", delimiter = ' ')
nal_17 = pd.read_table("data/DTL_NAL_2017.txt", delimiter = ' ')
nal_18 = pd.read_table("data/DTL_NAL_2018.txt", delimiter = ' ')
nal_19 = pd.read_table("data/DTL_NAL_2019.txt", delimiter = ' ')
enp_15 = pd.read_table("data/DTL_ENP_2015.txt", delimiter = ' ')
enp_16 = pd.read_table("data/DTL_ENP_2016.txt", delimiter = ' ')
enp_17 = pd.read_table("data/DTL_ENP_2017.txt", delimiter = ' ')
enp_18 = pd.read_table("data/DTL_ENP_2018.txt", delimiter = ' ')
enp_19 = pd.read_table("data/DTL_ENP_2019.txt", delimiter = ' ')
total = pd.concat([nal_15, nal_16, nal_17, nal_18, nal_19, enp_15, enp_16, enp_17, enp_18, enp_19], ignore_index = True)

total.columns = ['YEAR', "PART_ID", "NAME", "DATETIME", "LAT", "LON", "WIND", "PRESSURE", "CATEGORY", "DISTANCE"]

total['DATETIME'] = pd.to_datetime(total['DATETIME'], format = "%Y%m%d%H")
total['NAME'] = total['NAME'].str.upper()
name_convert = {'UNNAMED04': 'FOUR', 
                'UNNAMED11': 'ELEVEN',
                'UNNAMED08': 'EIGHT'}
total['NAME'].loc[total['NAME'].str.contains('UNNAMED')] = [name_convert[x] for x in total['NAME'].loc[total['NAME'].str.contains('UNNAMED')]]
total['ID'] = total['PART_ID'] + total['YEAR'].astype(str)
hurdat_all = h.Hurdat(data = total)
# Filter storms by minimum intensity of 50
hurdat = hurdat_all.genesis_to_lysis_filter(minimum_wind = 50)
# Add RI / RW labels to all storm observations
hurdat.identify_events(threshold = 25, col_names = ['25+', '25-'])
hurdat.identify_events(threshold = 15, col_names = ['15+', '15-'])
# Add 250 km from land restriction as a column
hurdat.distance_to_land_label(min_distance = 250)

/Users/galen/anaconda3/envs/ada/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [16]:
hurdat.storms['15-25'] = (hurdat.storms['15+'] & (~hurdat.storms['25+']))
hurdat.storms['m15-15'] = ~(hurdat.storms['25-'] | hurdat.storms['15-'] | hurdat.storms['15+'] | hurdat.storms['25+'])
hurdat.storms['m25-m15'] = (hurdat.storms['15-'] & (~hurdat.storms['25-']))

In [17]:
hurdat.drop_leading()

In [18]:
storm_data = hurdat.storms

In [19]:
jj = 0
storm_id = ''
storm_index = []
for ii in range(len(storm_data)):
    curr_id = storm_data['ID'][ii]
    if curr_id != storm_id:
        storm_id = curr_id
        jj = 0
        storm_index.append(jj)
    else:
        jj = jj + 1
        storm_index.append(jj)

In [20]:
storm_data['storm_index'] = storm_index

In [21]:
def get_ic(x):
    id = str(x['ID'])
    strm_id = str(x['storm_index'])
    root = 'data/int_circ_vortex/' # CHANGE IF YOU WANT VORTEX REMOVED 6/21
    ic = np.load(root + id + '_' + strm_id + '.npy')
    #return np.nanmean(ic)
    return np.nanmin(ic)

In [22]:
storm_data['ic'] = storm_data.apply(get_ic, axis = 1)

In [23]:
ids = pd.unique(storm_data['ID'])
data = pd.DataFrame(columns=['storm', 'wind', 'RI', 'shear'])
ii = 1
for id in ids:
    f = xr.open_dataset('data/profiles_vortex/prof_' + id + '.nc') # CHANGE IF YOU WANT VORTEX REMOVED 6/21
    data = data.append( pd.DataFrame( np.column_stack( ( ii*np.ones(f.wind.values.size), f.wind.values, f.ri.values, f.shear_mag.values ) ), columns=['storm', 'wind', 'RI', 'shear'] ), ignore_index = True )
    ii += 1

In [24]:
storm_data['shear'] = data['shear']

In [12]:
storm_data

,YEAR,PART_ID,NAME,DATETIME,LAT,LON,WIND,PRESSURE,CATEGORY,DISTANCE,...,25-,15+,15-,NEAR_LAND,15-25,m15-15,m25-m15,storm_index,ic,shear
0,2015,AL01,ANA,2015-05-09 00:00:00,31.9,-77.3,50,998,SS,229,...,False,False,False,True,False,True,False,0,0.137210,7.039532
1,2015,AL01,ANA,2015-05-09 06:00:00,32.2,-77.5,50,998,TS,194,...,False,False,False,True,False,True,False,1,0.069817,10.223293
2,2015,AL01,ANA,2015-05-09 12:00:00,32.5,-77.8,50,1001,TS,152,...,False,False,False,True,False,True,False,2,0.017772,11.931145
3,2015,AL02,BILL,2015-06-16 12:00:00,28.0,-96.0,50,997,TS,67,...,False,False,False,True,False,True,False,0,-0.236435,8.666265
4,2015,AL02,BILL,2015-06-16 18:00:00,28.2,-96.7,50,997,TS,8,...,False,False,False,True,False,True,False,1,-0.266149,8.549295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2552,2019,EP15,LORENA,2019-09-20 12:00:00,22.6,-108.8,65,989,HU,98,...,False,True,False,True,True,False,False,9,-0.328470,2.270408
2553,2019,EP15,LORENA,2019-09-20 18:00:00,23.0,-109.2,75,986,HU,38,...,False,True,True,True,True,False,True,10,-0.364039,2.131797
2554,2019,EP15,LORENA,2019-09-21 00:00:00,23.7,-109.5,70,987,HU,13,...,False,False,True,True,False,False,True,11,-0.356363,3.087351
2555,2019,EP15,LORENA,2019-09-21 06:00:00,24.2,-110.2,60,989,TS,0,...,False,False,True,True,False,False,True,12,-0.321296,2.762481


In [25]:
storm_data.to_csv('data/int_circ_min_summary_detailed_vortex.csv', index = False)